In [1]:
import numpy as np 
import pandas as pd
import os, zipfile

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


<h1>Preprocessing</h1>

In [2]:
# read in data
train_df = pd.read_csv('./train_onelabel.csv')
train_images_paths = ['./train_images/' + p for p in train_df.image]
train_labels = train_df['class'].values

In [3]:
# create image matrix
train_images = (load_img(p, target_size=(32, 32)) for p in train_images_paths)
train_images = [img_to_array(img)[:, :, [0]] for img in train_images]
train_images = np.array(train_images)
train_images.shape

(24204, 32, 32, 1)

In [4]:
# preprocessing of labels
classes = len(np.unique(train_labels))
train_labels = to_categorical(train_labels, classes)

In [5]:
# split data into test and train set
x_train, y_train, x_val,  y_val = train_test_split(train_images, train_labels, test_size=0.2)

In [6]:
# normalize image data

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_train /= 255
y_train /= 255

<h1>Model</h1>

In [7]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [8]:
# Initialising the CNN
classifier = Sequential()

In [9]:
# Step 1 - Convolution ---TODO: see input specification
classifier.add(Conv2D(32, (3, 3), input_shape = (32, 32,1),padding='same', activation = 'relu'))

In [10]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [11]:
# Adding a second convolutional layer+pooling layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [12]:
# Step 3 - Flattening
classifier.add(Flatten())

In [13]:
#Step 4 - Create a fully connected neural network
'''its good practise to pick a number of power of two-experimenting output_dim
128 hidden nodes in hidden layers-by experimenting'''
 

#output_dim = number of nodes in the hidden layer
classifier.add(Dense(units= 128, activation = 'relu'))

#Dropout (p was used in the old API, changed it to the new standard)
classifier.add(Dropout(rate = 0.1))

#second hidden layer
classifier.add(Dense(units = 128, activation = 'relu'))

#Dropout
#classifier.add(Dropout(p = 0.2))

#output layer
classifier.add(Dense(units = 121, activation = 'softmax')) #We have 121 categories of plankton!!

In [14]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [17]:
# fit model on training data
classifier.fit(x_train, x_val, batch_size=32, nb_epoch=10, verbose=1)

C:\Users\movetech\Anaconda2\envs\py35\lib\site-packages\keras\models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
19363/19363 [==============================] - 45s 2ms/step - loss: 3.4099 - acc: 0.2029
Epoch 2/10
19363/19363 [==============================] - 47s 2ms/step - loss: 2.5170 - acc: 0.3427
Epoch 3/10
19363/19363 [==============================] - 46s 2ms/step - loss: 2.2397 - acc: 0.3924
Epoch 4/10
19363/19363 [==============================] - 46s 2ms/step - loss: 2.0769 - acc: 0.4227
Epoch 5/10
19363/19363 [==============================] - 47s 2ms/step - loss: 1.9764 - acc: 0.4503
Epoch 6/10
19363/19363 [==============================] - 44s 2ms/step - loss: 1.8856 - acc: 0.4669
Epoch 7/10
19363/19363 [==============================] - 45s 2ms/step - loss: 1.8048 - acc: 0.4858
Epoch 8/10
19363/19363 [==============================] - 46s 2ms/step - loss: 1.7404 - acc: 0.4973
Epoch 9/10
19363/19363 [==============================] - 51s 3ms/step - loss: 1.6782 - acc: 0.5119
Epoch 10/10
19363/19363 [==============================] - 49s 3ms/step - loss: 1.6438 - acc: 0.5229

In [20]:
score = classifier.evaluate(y_train, y_val, verbose=0)

In [21]:
score

[1.7466102919039759, 0.52447841356938796]